In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [ ]:
train_df=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
submission_df=pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [ ]:
train_df.columns

In [ ]:
train_df.sample(5)

In [ ]:
test_df.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
submission_df.head(5)

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [ ]:
real_disaster=np.array(train_df[train_df['target']==1]['text'])
not_real_disaster=np.array(train_df[train_df['target']==0]['text'])

In [ ]:
real_disaster[10]

In [ ]:
not_real_disaster[10]

In [ ]:
from collections import Counter

In [ ]:
real_counts = Counter()
not_real_counts = Counter()
total_counts = Counter()

In [ ]:
for tweet in real_disaster:
    for word in tweet.split(" "):
        real_counts[word] +=1
        total_counts[word] +=1
    
for tweet in not_real_disaster:
    for word in tweet.split(" "):
        not_real_counts[word] +=1
        total_counts[word] +=1     

In [ ]:
real_counts.most_common()

In [ ]:
not_real_counts.most_common()

In [ ]:
ratio_counter=Counter()

for word,count in total_counts.most_common():
    if count>10:
        ratio_counter[word]=real_counts[word]/float(not_real_counts[word]+1)

for word,ratio in ratio_counter.most_common():
    if ratio>1:
        ratio_counter[word]=np.log(ratio)
    else:
        ratio_counter[word]=-np.log(1/(ratio+0.001))       


In [ ]:
ratio_counter.most_common()

In [ ]:
len(ratio_counter)

In [ ]:
vocab=[]
for key in ratio_counter.keys():
    if ratio_counter[key]>0.2 or ratio_counter[key]<-0.2:
        vocab.append(key)

vocab_size=len(vocab)
print(vocab_size)

1291


In [ ]:
layer_0=np.zeros((1,vocab_size))

In [ ]:
word2index={}

for idx,word in enumerate(vocab):
    word2index[word]=idx
word2index

{'the': 0,
 'a': 1,
 'to': 2,
 'in': 3,
 'and': 4,
 'I': 5,
 'for': 6,
 'is': 7,
 '': 8,
 'you': 9,
 'my': 10,
 'with': 11,
 'that': 12,
 'at': 13,
 'it': 14,
 'from': 15,
 'be': 16,
 'was': 17,
 'have': 18,
 'are': 19,
 'this': 20,
 'like': 21,
 '&amp;': 22,
 'A': 23,
 'just': 24,
 'your': 25,
 'up': 26,
 'but': 27,
 'so': 28,
 'me': 29,
 'not': 30,
 'has': 31,
 'out': 32,
 "I'm": 33,
 'will': 34,
 'via': 35,
 '??': 36,
 'after': 37,
 'an': 38,
 'about': 39,
 'get': 40,
 'or': 41,
 'when': 42,
 'all': 43,
 'no': 44,
 'into': 45,
 'over': 46,
 'In': 47,
 'who': 48,
 'we': 49,
 'fire': 50,
 'people': 51,
 'they': 52,
 '...': 53,
 'if': 54,
 '|': 55,
 'one': 56,
 'can': 57,
 'i': 58,
 'he': 59,
 'than': 60,
 'more': 61,
 "don't": 62,
 'do': 63,
 'what': 64,
 'his': 65,
 'would': 66,
 "it's": 67,
 'how': 68,
 'To': 69,
 'her': 70,
 'new': 71,
 'got': 72,
 'now': 73,
 'were': 74,
 'New': 75,
 'had': 76,
 '????': 77,
 'know': 78,
 'You': 79,
 'video': 80,
 'My': 81,
 'there': 82,
 'going': 

In [ ]:
def update_input_layer(tweet):
    
    global layer_0
    layer_0 *=0
    
    for word in tweet.split(" "):
        if word in vocab:
            layer_0[0][word2index[word]]+=1
    return layer_0

In [ ]:
real_disaster[0]

In [ ]:
update_input_layer(real_disaster[0])
layer_0

In [ ]:
from sklearn import model_selection
traindf=train_df[['text','target']]
train,validation=model_selection.train_test_split(traindf,test_size=0.2, random_state=42, shuffle=True)

In [ ]:
import torch
from torch import nn
from torch import optim
#import torch.nn.functional as F

In [ ]:
# class network(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.hidden=nn.Linear(111,256)
#         self.output=nn.Linear(256,2)
        
#     def forward(self,x):
#         x=F.relu(self.hidden(x))
#         x=F.sigmoid(self.output(x))
        
#     return x
    

In [ ]:
model=nn.Sequential(nn.Linear(vocab_size,500),nn.ReLU(),nn.Linear(500,200),nn.ReLU(),nn.Linear(200,2),nn.Sigmoid())
#model=nn.Sequential(nn.Linear(vocab_size,300),nn.ReLU(),nn.Linear(300,2),nn.Sigmoid())
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.03)

In [ ]:
data=train[['text','target']].values
t=0
for tweet,target in data:
    tweet=torch.tensor(update_input_layer(tweet))
    model_=model(tweet.float())
    loss=criterion(model_,torch.tensor([target]))
#     if t % 100 == 99:
#         print(model_)
#         print(t, loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    t+=1
            

In [ ]:
data=validation[['text','target']].values
with torch.no_grad():
    ct=0
    correct=0
    for tweet,target in data:
        tweet=torch.tensor(update_input_layer(tweet))
        model_=model(tweet.float())
        if (target-model_[0][1]<0.4) and (target-model_[0][1]>-0.4):
            correct+=1
        ct+=1
    print('correct {} ,ct {} ,accuracy {} '.format(correct,ct,correct/float(ct)))

correct 974 ,ct 1523 ,accuracy 0.639527248850952 


In [ ]:
def get_result(tweet):
    with torch.no_grad():
        tweet=torch.tensor(update_input_layer(tweet))
        model_=model(tweet.float())
        if model_[0][1]>0.6:
            target=1
        else:target=0
        return target
            
submission_df['target']=test_df['text'].apply(get_result)

In [ ]:
submission_df.to_csv("")

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,1
...,...,...
3258,10861,0
3259,10865,0
3260,10868,1
3261,10874,0
